Parts of code Refereed from https://mccormickml.com/2019/07/22/BERT-fine-tuning/#34-training--validation-split

# Check GPU

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


Assign GPU

In [ ]:
import torch

# Check if GPU is available then telling PyTorch to use the GPU 
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# Make use of CPU
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Pip Install

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 25.6MB/s 
     |████████████████████████████████| 2.9MB 49.6MB/s 
     |████████████████████████████████| 890kB 49.9MB/s 
     |████████████████████████████████| 1.1MB 46.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4ca69027d3c08df11bda46e1b3f2c9951163ca69b3b767fed4f87f52b4820a7e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


#Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


Imports

In [ ]:
import pandas as pd
import re

#Load Dataset

In [ ]:
import re
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Data/Combined_FAANG_binary_previous.csv', sep=',')

In [ ]:
df

,symbol,message,datetime,user,message_id,Date,Time,label
0,AAPL,qq next 60min confirm start rally aapl coming ...,2015-12-21 18:37:24,191996.0,47148173.0,2015-12-21,18:37:24,1
1,AAPL,aapl watching gap fill 169 20,2018-11-24 07:02:32,1665234.0,146068732.0,2018-11-24,07:02:32,1
2,AAPL,aapl weekly options gamblers lose,2014-07-22 21:48:13,71738.0,24904954.0,2014-07-22,21:48:13,1
3,AAPL,aapl,2020-01-27 07:07:03,1229493.0,191978042.0,2020-01-27,07:07:03,0
4,AAPL,key levels watch aapl,2014-06-27 15:19:47,106412.0,24190263.0,2014-06-27,15:19:47,1
...,...,...,...,...,...,...,...,...
2566853,NFLX,nflx sister owns kinda thinking telling sell b...,2019-01-11 20:51:22,607557.0,150426203.0,2019-01-11,20:51:22,1
2566854,NFLX,nflx bought 123 shares think hit 175 tomorrow,2017-07-17 19:34:14,453320.0,89017330.0,2017-07-17,19:34:14,1
2566855,NFLX,quot vsfinancials quot investcorrectly netflix...,2015-06-10 13:02:32,419879.0,38055272.0,2015-06-10,13:02:32,1
2566856,NFLX,mgt 32 million volume aapl 41 nflx 11 msft 8 31,2016-05-12 15:37:16,248795.0,54671454.0,2016-05-12,15:37:16,0


In [ ]:
# df['polarity'].isnan()

Limiting the dataset time-line (Change date for 1 year or 2 year)

In [ ]:
df.drop(df[df['Date'] <= '2018-07-20'].index, inplace = True)
df


,symbol,message,datetime,user,message_id,Date,Time,label
1,AAPL,aapl watching gap fill 169 20,2018-11-24 07:02:32,1665234.0,146068732.0,2018-11-24,07:02:32,1
3,AAPL,aapl,2020-01-27 07:07:03,1229493.0,191978042.0,2020-01-27,07:07:03,0
5,AAPL,aapl loads cash hand great service business lo...,2018-11-01 23:39:14,123291.0,143688765.0,2018-11-01,23:39:14,1
8,AAPL,qq became euphoric calls exp week aiming ath f...,2020-05-13 02:13:00,2250451.0,212222428.0,2020-05-13,02:13:00,0
9,AAPL,spy novices like davey day trader lose money s...,2020-06-24 11:12:09,543250.0,222404886.0,2020-06-24,11:12:09,0
...,...,...,...,...,...,...,...,...
2566843,NFLX,nflx last 3 months big green candles followed ...,2019-05-01 18:21:25,637003.0,162589986.0,2019-05-01,18:21:25,1
2566848,NFLX,nflx full move key support,2020-03-12 16:52:43,677915.0,199933357.0,2020-03-12,16:52:43,0
2566849,NFLX,spy spx nflx nvda virtually volume today absen...,2019-10-14 18:16:28,55818.0,180328889.0,2019-10-14,18:16:28,1
2566853,NFLX,nflx sister owns kinda thinking telling sell b...,2019-01-11 20:51:22,607557.0,150426203.0,2019-01-11,20:51:22,1


In [ ]:
df = df.sample(frac=1)

Replacing value of Negative label from -1 to 0

In [ ]:
df.drop(df[df['label'] == 0].index, inplace = True) 
df["label"].replace({-1: 0}, inplace=True) 
# 0 - Neutral, 1 - positive, 2 - Negative - 3 lables
# 0 - Negative, 1 - positive

#Splitting Dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'].values, df['label'].values, test_size=0.1, random_state=42)
# df_train = pd.DataFrame(X_train,columns=['message'])
# df_train

In [ ]:
df_train = pd.DataFrame(y_train)
df_train.value_counts()

1    493779
0    425526
dtype: int64

In [ ]:
df_test = pd.DataFrame(y_test)
df_test.value_counts()

1    55115
0    47030
dtype: int64

In [ ]:
X_train

array(['aapl lot apple feels like shit duck let us call like going plummet nose ducking dive bedrock metrics show one thing sure even repeated drops stocks simply gone outrageously overpriced overpriced coronavirus catalyst kicked current cycle doubt another factor play 2019 stock prices roared ahead earnings stalled creating mismatch inflated valuations dependent rising profits profits hit wall says franz wall grown higher earnings expectations amp p 500 already muted come given potential supply disruptions caused coronavirus outbreak',
       'ba aapl amzn shop beware market makers manipulation open',
       'amzn speaking amzn aapl er tap next week big tech house hearings sched begin mon bezos cook expected testify coincidence sarcasm',
       ..., 'aapl ah always good sign goes tmr',
       'aapl jpm bac month ago strongest econ ever recession games jamie dimon needs step like 2016',
       'amzn 1843 might waterfall'], dtype=object)

In [ ]:
sentences = X_train
labels = y_train

#BERT Model 

### Bert tokenizer

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Changing Text to the format required for BERT model training

In [ ]:
input_ids = []
attention_masks = []

# For each sentence encode_plus would tokenize, add special tokens ([CLS], [SEP]), map tokens to IDs, create attention masks
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True,
                        max_length = 160,         
                        truncation = True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Converting the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  aapl bulls need chew red candle 210
Token IDs: tensor([  101,  9779, 24759, 12065,  2342, 21271,  2417, 13541, 12875,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

# Creating TensorDataset by combining the training inputs
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Dividing the training and validation dataset by selecting samples randomly
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{} training samples'.format(train_size))
print('{} validation samples'.format(val_size))



827374 training samples
91931 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [ ]:
from transformers import BertModel, BertForSequenceClassification, AdamW, BertConfig


# Loading BertForSequenceClassification model for training
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2, #change here if classes changed to 3   
    output_attentions = False,
    output_hidden_states = False, 
)

torch.cuda.empty_cache()

# Assigning the model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Printing the Model architecture

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

#Fine Tune

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, 
                  eps = 1e-8,
                  weight_decay = 1e-5
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np
# Function for calculating the accuracy 
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    
    #Setting the model to training mode
    model.train()

    # This block is for progress update of every batch 
    for step, batch in enumerate(train_dataloader):

        
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # From the dataloader unpacking the training batch  
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        
        #model outputs
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)


        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)            
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
 
    print("")
    print("Running Validation...")

    t0 = time.time()

    # Setting the model to evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

 
    for batch in validation_dataloader:
        
        # Unpack this validation batch from our dataloader. 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        

            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch   100  of  25,856.    Elapsed: 0:00:51.
  Batch   200  of  25,856.    Elapsed: 0:01:41.
  Batch   300  of  25,856.    Elapsed: 0:02:32.
  Batch   400  of  25,856.    Elapsed: 0:03:22.
  Batch   500  of  25,856.    Elapsed: 0:04:13.
  Batch   600  of  25,856.    Elapsed: 0:05:03.
  Batch   700  of  25,856.    Elapsed: 0:05:54.
  Batch   800  of  25,856.    Elapsed: 0:06:44.
  Batch   900  of  25,856.    Elapsed: 0:07:35.
  Batch 1,000  of  25,856.    Elapsed: 0:08:25.
  Batch 1,100  of  25,856.    Elapsed: 0:09:16.
  Batch 1,200  of  25,856.    Elapsed: 0:10:06.
  Batch 1,300  of  25,856.    Elapsed: 0:10:57.
  Batch 1,400  of  25,856.    Elapsed: 0:11:47.
  Batch 1,500  of  25,856.    Elapsed: 0:12:38.
  Batch 1,600  of  25,856.    Elapsed: 0:13:28.
  Batch 1,700  of  25,856.    Elapsed: 0:14:19.
  Batch 1,800  of  25,856.    Elapsed: 0:15:09.
  Batch 1,900  of  25,856.    Elapsed: 0:16:00.
  Batch 2,000  of  25,856.    Elapsed: 0:16:5

#Saving Fine-tuned Model

In [ ]:
import os
#defining output directory

output_dir = '/content/drive/MyDrive/BERT-Final/2Year_Binary/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to /content/drive/MyDrive/BERT-Final/2Year_Binary/


('/content/drive/MyDrive/BERT-Final/2Year_Binary/tokenizer_config.json',
 '/content/drive/MyDrive/BERT-Final/2Year_Binary/special_tokens_map.json',
 '/content/drive/MyDrive/BERT-Final/2Year_Binary/vocab.txt',
 '/content/drive/MyDrive/BERT-Final/2Year_Binary/added_tokens.json')

In [ ]:
model_save_name = 'bert_model_2year.bin'
path = F"/content/drive/MyDrive/BERT-Final/2Year_Binary/"+model_save_name
torch.save(model.state_dict(), path)

###Testing

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
from transformers import BertModel, BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer

output_dir = '/content/drive/MyDrive/BERT-Final/2Year_Binary/'

model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
labels = y_test

In [ ]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

print('Number of test sentences: {:,}\n'.format(df_test.shape[0]))

# Create sentence and label lists
test_sentences = X_test
test_labels = y_test

input_ids = []
attention_masks = []

# For each sentence encode_plus would tokenize, add special tokens ([CLS], [SEP]), map tokens to IDs, create attention masks
for sent in test_sentences:
   
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 160,          
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt',     
                   )
    
   
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
batch_size = 32  
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Number of test sentences: 102,145



/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
model.eval()
predictions , true_labels = [], []
t0 = time.time()
for step, batch in enumerate(prediction_dataloader):
  if step % 100 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(prediction_dataloader), elapsed))

  batch = tuple(t for t in batch)
  
 
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 102,145 test sentences...
  Batch   100  of  3,193.    Elapsed: 0:00:28.
  Batch   200  of  3,193.    Elapsed: 0:00:58.
  Batch   300  of  3,193.    Elapsed: 0:01:29.
  Batch   400  of  3,193.    Elapsed: 0:02:00.
  Batch   500  of  3,193.    Elapsed: 0:02:32.
  Batch   600  of  3,193.    Elapsed: 0:03:06.
  Batch   700  of  3,193.    Elapsed: 0:03:39.
  Batch   800  of  3,193.    Elapsed: 0:04:12.
  Batch   900  of  3,193.    Elapsed: 0:04:45.
  Batch 1,000  of  3,193.    Elapsed: 0:05:18.
  Batch 1,100  of  3,193.    Elapsed: 0:05:51.
  Batch 1,200  of  3,193.    Elapsed: 0:06:24.
  Batch 1,300  of  3,193.    Elapsed: 0:06:57.
  Batch 1,400  of  3,193.    Elapsed: 0:07:30.
  Batch 1,500  of  3,193.    Elapsed: 0:08:03.
  Batch 1,600  of  3,193.    Elapsed: 0:08:36.
  Batch 1,700  of  3,193.    Elapsed: 0:09:09.
  Batch 1,800  of  3,193.    Elapsed: 0:09:42.
  Batch 1,900  of  3,193.    Elapsed: 0:10:15.
  Batch 2,000  of  3,193.    Elapsed: 0:10:48.
  Batch 2,10

#Classification Report 

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

print("F1 score: {}".format(precision_recall_fscore_support(flat_predictions, flat_true_labels, average='macro')))

F1 score: (0.5933196066624191, 0.6043964133974533, 0.5883778879480615, None)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(flat_true_labels, flat_predictions,zero_division=1))

              precision    recall  f1-score   support

           0       0.60      0.43      0.50     47030
           1       0.61      0.75      0.67     55115

    accuracy                           0.61    102145
   macro avg       0.60      0.59      0.59    102145
weighted avg       0.60      0.61      0.60    102145

